In [1]:
"""# 🔧 INSTALACIÓN DE DEPENDENCIAS (ejecutar solo UNA vez)
import sys
import subprocess

print("🔄 Instalando PyTorch CPU-only y tf-keras...")
subprocess.check_call([
    sys.executable, '-m', 'pip', 'install', 
    'torch', '--index-url', 'https://download.pytorch.org/whl/cpu',
    '-q'
])
subprocess.check_call([
    sys.executable, '-m', 'pip', 'install', 'tf-keras', '-q'
])
print("✅ PyTorch y tf-keras instalados correctamente")
print("⚠️  IMPORTANTE: Después de ejecutar esta celda, puedes eliminarla o comentarla")"""

'# 🔧 INSTALACIÓN DE DEPENDENCIAS (ejecutar solo UNA vez)\nimport sys\nimport subprocess\n\nprint("🔄 Instalando PyTorch CPU-only y tf-keras...")\nsubprocess.check_call([\n    sys.executable, \'-m\', \'pip\', \'install\', \n    \'torch\', \'--index-url\', \'https://download.pytorch.org/whl/cpu\',\n    \'-q\'\n])\nsubprocess.check_call([\n    sys.executable, \'-m\', \'pip\', \'install\', \'tf-keras\', \'-q\'\n])\nprint("✅ PyTorch y tf-keras instalados correctamente")\nprint("⚠️  IMPORTANTE: Después de ejecutar esta celda, puedes eliminarla o comentarla")'

## 1️⃣ Importación de Librerías

In [2]:
# Manipulación de datos
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# NLP y Transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Configuración
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Configurar dispositivo (usar CPU)
device = -1  # -1 = CPU en transformers pipeline
print(f"🖥️ Dispositivo configurado: CPU")
print(f"🔧 PyTorch versión: {torch.__version__}")
print("✅ Librerías importadas correctamente")

/home/els4nchez/Videos/TECH/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-02 23:05:30.390932: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-02 23:05:31.209682: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-02 23:05:30.390932: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-02 23:05:31.209682: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use 

🖥️ Dispositivo configurado: CPU
🔧 PyTorch versión: 2.9.1+cpu
✅ Librerías importadas correctamente


---

## 2️⃣ Carga de Datos

In [3]:
# Configurar rutas del proyecto
BASE_DIR = Path('/home/els4nchez/Videos/TECH')
DATA_DIR = BASE_DIR / 'data'
UNIFICACION_DIR = BASE_DIR / 'unificacion'
DATA_PROCESADO_DIR = UNIFICACION_DIR / 'datos_procesados'
FIGURAS_DIR = UNIFICACION_DIR / 'figuras'

# Crear directorio de figuras si no existe
FIGURAS_DIR.mkdir(parents=True, exist_ok=True)

# Cargar noticias del EDA
ruta_noticias = DATA_PROCESADO_DIR / 'noticias_oro_limpias.csv'

if not ruta_noticias.exists():
    print("❌ ERROR: Archivo no encontrado.")
    print("⚠️  Debes ejecutar primero el notebook 01_Introduccion_y_Carga_de_Datos.ipynb")
    raise FileNotFoundError(f"No se encontró: {ruta_noticias}")

# Cargar datos
df_noticias = pd.read_csv(ruta_noticias, parse_dates=['fecha'])

print("✅ Datos cargados exitosamente")
print(f"   Total de noticias: {len(df_noticias):,}")
print(f"   Rango temporal: {df_noticias['fecha'].min().date()} → {df_noticias['fecha'].max().date()}")


✅ Datos cargados exitosamente
   Total de noticias: 18,776
   Rango temporal: 2016-01-03 → 2025-10-31


---

## 3️⃣ Carga del Modelo FinBERT

**FinBERT** (ProsusAI/finbert):
- Base: BERT pre-entrenado en textos financieros
- Especialización: Análisis de sentimientos en contexto financiero
- Idioma: Inglés
- Clases: positive, negative, neutral
- Paper: "FinBERT: Financial Sentiment Analysis with Pre-trained Language Models" (Araci, 2019)

In [4]:
# Cargar modelo FinBERT
print("📥 Descargando modelo FinBERT...")
print("   Esto puede tardar unos minutos en la primera ejecución.")
print("   El modelo se almacenará en caché para ejecuciones futuras.\n")

model_name = "ProsusAI/finbert"

try:
    # Crear pipeline de análisis de sentimientos
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model=model_name,
        tokenizer=model_name,
        device=device,
        max_length=512,
        truncation=True
    )
    
    print("✅ Modelo FinBERT cargado exitosamente")
    print(f"   Modelo: {model_name}")
    print(f"   Dispositivo: {'GPU' if device == 0 else 'CPU'}")
    print(f"   Clases: positive, negative, neutral")
    
except Exception as e:
    print(f"❌ Error al cargar modelo: {e}")
    print("\n⚠️  Solución alternativa: Intentando con modelo más ligero...")
    
    # Fallback a modelo alternativo si FinBERT falla
    model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model=model_name,
        device=device,
        max_length=512,
        truncation=True
    )
    print(f"✅ Modelo alternativo cargado: {model_name}")

📥 Descargando modelo FinBERT...
   Esto puede tardar unos minutos en la primera ejecución.
   El modelo se almacenará en caché para ejecuciones futuras.



Device set to use cpu


✅ Modelo FinBERT cargado exitosamente
   Modelo: ProsusAI/finbert
   Dispositivo: CPU
   Clases: positive, negative, neutral


In [5]:
# Probar el modelo con ejemplos
print("🧪 PRUEBA DEL MODELO CON EJEMPLOS")
print("="*70)

test_examples = [
    "Gold prices surge to record high amid economic uncertainty",
    "Gold falls as dollar strengthens and investors flee safe havens",
    "Gold prices remain stable in quiet trading session",
    "Central bank increases gold reserves signaling confidence",
    "Market turmoil drives gold demand to lowest in decade"
]

print("\nEjemplos de clasificación:\n")
for i, text in enumerate(test_examples, 1):
    result = sentiment_pipeline(text)[0]
    label = result['label'].upper()
    score = result['score']
    print(f"{i}. {text}")
    print(f"   → Sentimiento: {label} (confianza: {score:.3f})\n")

🧪 PRUEBA DEL MODELO CON EJEMPLOS

Ejemplos de clasificación:

1. Gold prices surge to record high amid economic uncertainty
   → Sentimiento: POSITIVE (confianza: 0.834)

2. Gold falls as dollar strengthens and investors flee safe havens
   → Sentimiento: NEGATIVE (confianza: 0.724)

3. Gold prices remain stable in quiet trading session
   → Sentimiento: POSITIVE (confianza: 0.550)

4. Central bank increases gold reserves signaling confidence
   → Sentimiento: POSITIVE (confianza: 0.904)

5. Market turmoil drives gold demand to lowest in decade
   → Sentimiento: NEGATIVE (confianza: 0.959)

3. Gold prices remain stable in quiet trading session
   → Sentimiento: POSITIVE (confianza: 0.550)

4. Central bank increases gold reserves signaling confidence
   → Sentimiento: POSITIVE (confianza: 0.904)

5. Market turmoil drives gold demand to lowest in decade
   → Sentimiento: NEGATIVE (confianza: 0.959)



---

## 4️⃣ Análisis de Sentimientos - Procesamiento por Lotes

In [6]:
def analyze_sentiment_batch(texts, batch_size=16, show_progress=True):
    """
    Analiza sentimientos de textos en lotes para mayor eficiencia.
    
    Parameters:
    -----------
    texts : list
        Lista de textos a analizar
    batch_size : int
        Tamaño del lote para procesamiento paralelo
    show_progress : bool
        Mostrar barra de progreso
    
    Returns:
    --------
    list : Lista de diccionarios con sentimiento y score
    """
    results = []
    
    # Procesar en lotes
    if show_progress:
        iterator = tqdm(range(0, len(texts), batch_size), desc="Analizando sentimientos")
    else:
        iterator = range(0, len(texts), batch_size)
    
    for i in iterator:
        batch = texts[i:i + batch_size]
        batch_results = sentiment_pipeline(batch)
        results.extend(batch_results)
    
    return results

# Preparar textos
print("🔄 Preparando textos para análisis...")
textos_analizar = df_noticias['titulo'].tolist()
print(f"   Total de títulos a analizar: {len(textos_analizar):,}")
print(f"\n⏳ Iniciando análisis de sentimientos...")
print(f"   Esto puede tardar varios minutos dependiendo del hardware.\n")

🔄 Preparando textos para análisis...
   Total de títulos a analizar: 18,776

⏳ Iniciando análisis de sentimientos...
   Esto puede tardar varios minutos dependiendo del hardware.



In [7]:
# Ejecutar análisis de sentimientos
sentimiento_results = analyze_sentiment_batch(
    textos_analizar,
    batch_size=16,
    show_progress=True
)

print("\n✅ Análisis completado")
print(f"   Resultados obtenidos: {len(sentimiento_results):,}")

Analizando sentimientos: 100%|██████████| 1174/1174 [09:48<00:00,  2.00it/s]


✅ Análisis completado
   Resultados obtenidos: 18,776


In [8]:
# Extraer etiquetas y scores
df_noticias['sentiment_label'] = [r['label'] for r in sentimiento_results]
df_noticias['sentiment_score'] = [r['score'] for r in sentimiento_results]

# Mapear labels a español para interpretación
label_map = {
    'positive': 'Positivo',
    'negative': 'Negativo',
    'neutral': 'Neutral',
    'POSITIVE': 'Positivo',
    'NEGATIVE': 'Negativo',
    'NEUTRAL': 'Neutral',
    'LABEL_0': 'Negativo',  # Para algunos modelos alternativos
    'LABEL_1': 'Neutral',
    'LABEL_2': 'Positivo'
}

df_noticias['sentiment'] = df_noticias['sentiment_label'].map(label_map)

print("✅ Resultados integrados al DataFrame")
print(f"\n📊 Primeras 5 noticias con sentimiento:")
print(df_noticias[['fecha', 'titulo', 'sentiment', 'sentiment_score']].head())

✅ Resultados integrados al DataFrame

📊 Primeras 5 noticias con sentimiento:
       fecha                                             titulo sentiment  \
0 2016-01-03  Fischer: Fed’s Tools ‘Proved Effective’ in Rai...  Positivo   
1 2016-01-03     The Fed’s Interest-Rate Path: Who Has the Map?   Neutral   
2 2016-01-03  Indonesia Meets Inflation Goal, Central Bank A...  Positivo   
3 2016-01-03  Fed Officials, Economists Struggle With Key In...  Negativo   
4 2016-01-03  Fed Was ‘Prudent’ To Raise Rates In December, ...   Neutral   

   sentiment_score  
0         0.913104  
1         0.911072  
2         0.601469  
3         0.822698  
4         0.531945  


---

## 5️⃣ Análisis Estadístico de Sentimientos

In [9]:
# Distribución de sentimientos
print("📊 DISTRIBUCIÓN DE SENTIMIENTOS")
print("="*70)

sentiment_counts = df_noticias['sentiment'].value_counts()
sentiment_percentages = df_noticias['sentiment'].value_counts(normalize=True) * 100

for sentiment in ['Positivo', 'Neutral', 'Negativo']:
    if sentiment in sentiment_counts.index:
        count = sentiment_counts[sentiment]
        pct = sentiment_percentages[sentiment]
        print(f"   {sentiment:10s}: {count:5d} noticias ({pct:5.2f}%)")

# Estadísticas de scores por sentimiento
print(f"\n📈 Estadísticas de Confianza (Score) por Sentimiento:")
print("="*70)
print(df_noticias.groupby('sentiment')['sentiment_score'].describe())

📊 DISTRIBUCIÓN DE SENTIMIENTOS
   Positivo  :  4085 noticias (21.76%)
   Neutral   :  8007 noticias (42.64%)
   Negativo  :  6684 noticias (35.60%)

📈 Estadísticas de Confianza (Score) por Sentimiento:
            count      mean       std       min       25%       50%       75%  \
sentiment                                                                       
Negativo   6684.0  0.793868  0.157396  0.335222  0.678427  0.846356  0.929387   
Neutral    8007.0  0.784010  0.142149  0.338438  0.696164  0.832142  0.901158   
Positivo   4085.0  0.740880  0.154915  0.344163  0.614777  0.768987  0.882012   

                max  
sentiment            
Negativo   0.976285  
Neutral    0.954514  
Positivo   0.959062  


In [10]:
# Gráfico de barras de distribución
fig = go.Figure()

colors = {'Positivo': 'green', 'Neutral': 'gray', 'Negativo': 'red'}
sentiment_order = ['Positivo', 'Neutral', 'Negativo']

for sentiment in sentiment_order:
    if sentiment in sentiment_counts.index:
        fig.add_trace(go.Bar(
            x=[sentiment],
            y=[sentiment_counts[sentiment]],
            name=sentiment,
            marker_color=colors[sentiment],
            text=[sentiment_counts[sentiment]],  # Convertir a lista
            textposition='outside'
        ))

fig.update_layout(
    title='📊 Distribución de Sentimientos - Noticias WSJ sobre Oro',
    xaxis_title='Sentimiento',
    yaxis_title='Número de Noticias',
    template='plotly_white',
    height=500,
    showlegend=False
)

fig.show()

In [11]:
# Distribución de scores de confianza
fig = go.Figure()

for sentiment in sentiment_order:
    if sentiment in df_noticias['sentiment'].values:
        data = df_noticias[df_noticias['sentiment'] == sentiment]['sentiment_score']
        fig.add_trace(go.Box(
            y=data,
            name=sentiment,
            marker_color=colors[sentiment]
        ))

fig.update_layout(
    title='📊 Distribución de Scores de Confianza por Sentimiento',
    xaxis_title='Sentimiento',
    yaxis_title='Score de Confianza',
    template='plotly_white',
    height=500
)

fig.show()

---

## 6️⃣ Análisis Temporal de Sentimientos

In [12]:
# Crear variable numérica de sentimiento
# Positivo = +1, Neutral = 0, Negativo = -1
sentiment_numeric_map = {
    'Positivo': 1,
    'Neutral': 0,
    'Negativo': -1
}

df_noticias['sentiment_numeric'] = df_noticias['sentiment'].map(sentiment_numeric_map)

# También crear versión ponderada por confianza
df_noticias['sentiment_weighted'] = df_noticias['sentiment_numeric'] * df_noticias['sentiment_score']

print("✅ Variables numéricas de sentimiento creadas")
print(f"\n📊 Ejemplos:")
print(df_noticias[['fecha', 'titulo', 'sentiment', 'sentiment_numeric', 'sentiment_weighted']].head())

✅ Variables numéricas de sentimiento creadas

📊 Ejemplos:
       fecha                                             titulo sentiment  \
0 2016-01-03  Fischer: Fed’s Tools ‘Proved Effective’ in Rai...  Positivo   
1 2016-01-03     The Fed’s Interest-Rate Path: Who Has the Map?   Neutral   
2 2016-01-03  Indonesia Meets Inflation Goal, Central Bank A...  Positivo   
3 2016-01-03  Fed Officials, Economists Struggle With Key In...  Negativo   
4 2016-01-03  Fed Was ‘Prudent’ To Raise Rates In December, ...   Neutral   

   sentiment_numeric  sentiment_weighted  
0                  1            0.913104  
1                  0            0.000000  
2                  1            0.601469  
3                 -1           -0.822698  
4                  0            0.000000  


In [13]:
# Agregación diaria de sentimientos
df_noticias['fecha_date'] = df_noticias['fecha'].dt.date

sentimiento_diario = df_noticias.groupby('fecha_date').agg({
    'sentiment_numeric': 'mean',
    'sentiment_weighted': 'mean',
    'sentiment_score': 'mean',
    'titulo': 'count'
}).rename(columns={'titulo': 'num_noticias'})

sentimiento_diario.index = pd.to_datetime(sentimiento_diario.index)

# Calcular distribución de sentimientos por día
sentiment_daily_dist = df_noticias.groupby(['fecha_date', 'sentiment']).size().unstack(fill_value=0)
sentiment_daily_dist.index = pd.to_datetime(sentiment_daily_dist.index)

print("✅ Agregación diaria completada")
print(f"\n📊 Sentimiento promedio diario (primeras 5 fechas):")
print(sentimiento_diario.head())

✅ Agregación diaria completada

📊 Sentimiento promedio diario (primeras 5 fechas):
            sentiment_numeric  sentiment_weighted  sentiment_score  \
fecha_date                                                           
2016-01-03           0.083333            0.036220         0.813113   
2016-01-04           0.153846            0.104399         0.835821   
2016-01-05          -0.263158           -0.226055         0.776322   
2016-01-06          -0.315789           -0.256826         0.769067   
2016-01-08           0.166667            0.133947         0.797638   

            num_noticias  
fecha_date                
2016-01-03            12  
2016-01-04            13  
2016-01-05            19  
2016-01-06            19  
2016-01-08            18  


In [14]:
# Serie temporal de sentimiento promedio
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    subplot_titles=('Sentimiento Promedio Diario', 'Número de Noticias por Día'),
    row_heights=[0.7, 0.3]
)

# Subplot 1: Sentimiento
fig.add_trace(go.Scatter(
    x=sentimiento_diario.index,
    y=sentimiento_diario['sentiment_weighted'],
    mode='lines',
    name='Sentimiento Ponderado',
    line=dict(color='blue', width=1.5),
    fill='tozeroy',
    fillcolor='rgba(0,100,255,0.2)'
), row=1, col=1)

# Línea de referencia en 0
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=1)

# Subplot 2: Volumen de noticias
fig.add_trace(go.Bar(
    x=sentimiento_diario.index,
    y=sentimiento_diario['num_noticias'],
    name='Noticias',
    marker_color='lightcoral'
), row=2, col=1)

fig.update_xaxes(title_text="Fecha", row=2, col=1)
fig.update_yaxes(title_text="Sentimiento (-1 a +1)", row=1, col=1)
fig.update_yaxes(title_text="Nº Noticias", row=2, col=1)

fig.update_layout(
    title_text="📈 Evolución Temporal del Sentimiento de Noticias",
    height=700,
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

print("💡 Interpretación:")
print("   - Valores > 0: Sentimiento predominantemente positivo")
print("   - Valores < 0: Sentimiento predominantemente negativo")
print("   - Valores ≈ 0: Sentimiento neutral o mixto")

💡 Interpretación:
   - Valores > 0: Sentimiento predominantemente positivo
   - Valores < 0: Sentimiento predominantemente negativo
   - Valores ≈ 0: Sentimiento neutral o mixto


In [15]:
# Media móvil del sentimiento (suavizar señal)
sentimiento_diario['sentiment_ma_7'] = sentimiento_diario['sentiment_weighted'].rolling(window=7, center=True).mean()
sentimiento_diario['sentiment_ma_30'] = sentimiento_diario['sentiment_weighted'].rolling(window=30, center=True).mean()

fig = go.Figure()

# Sentimiento diario (raw)
fig.add_trace(go.Scatter(
    x=sentimiento_diario.index,
    y=sentimiento_diario['sentiment_weighted'],
    mode='lines',
    name='Diario',
    line=dict(color='lightgray', width=1),
    opacity=0.5
))

# MA 7 días
fig.add_trace(go.Scatter(
    x=sentimiento_diario.index,
    y=sentimiento_diario['sentiment_ma_7'],
    mode='lines',
    name='MA(7)',
    line=dict(color='blue', width=2)
))

# MA 30 días
fig.add_trace(go.Scatter(
    x=sentimiento_diario.index,
    y=sentimiento_diario['sentiment_ma_30'],
    mode='lines',
    name='MA(30)',
    line=dict(color='darkblue', width=2.5)
))

fig.add_hline(y=0, line_dash="dash", line_color="gray")

fig.update_layout(
    title='📈 Sentimiento con Medias Móviles (7 y 30 días)',
    xaxis_title='Fecha',
    yaxis_title='Sentimiento Ponderado',
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()

---

## 7️⃣ Identificación de Eventos de Sentimiento Extremo

In [16]:
# Días con sentimiento más positivo
print("🟢 TOP 10 DÍAS CON SENTIMIENTO MÁS POSITIVO")
print("="*70)

top_positive = sentimiento_diario.nlargest(10, 'sentiment_weighted')
print(top_positive[['sentiment_weighted', 'num_noticias', 'sentiment_score']])

# Días con sentimiento más negativo
print("\n🔴 TOP 10 DÍAS CON SENTIMIENTO MÁS NEGATIVO")
print("="*70)

top_negative = sentimiento_diario.nsmallest(10, 'sentiment_weighted')
print(top_negative[['sentiment_weighted', 'num_noticias', 'sentiment_score']])

🟢 TOP 10 DÍAS CON SENTIMIENTO MÁS POSITIVO
            sentiment_weighted  num_noticias  sentiment_score
fecha_date                                                   
2021-01-28            0.948114             1         0.948114
2020-01-20            0.940170             1         0.940170
2016-01-17            0.933800             1         0.933800
2019-06-02            0.928998             1         0.928998
2020-11-22            0.898987             1         0.898987
2023-07-15            0.884765             1         0.884765
2025-04-19            0.884334             1         0.884334
2021-05-14            0.880637             1         0.880637
2021-02-15            0.876587             1         0.876587
2024-12-28            0.869756             1         0.869756

🔴 TOP 10 DÍAS CON SENTIMIENTO MÁS NEGATIVO
            sentiment_weighted  num_noticias  sentiment_score
fecha_date                                                   
2018-08-19           -0.969650             1 

In [17]:
# Identificar días con sentimiento extremo (fuera de ±1.5 std)
sentiment_mean = sentimiento_diario['sentiment_weighted'].mean()
sentiment_std = sentimiento_diario['sentiment_weighted'].std()

threshold_positive = sentiment_mean + 1.5 * sentiment_std
threshold_negative = sentiment_mean - 1.5 * sentiment_std

sentimiento_diario['sentiment_extreme'] = (
    (sentimiento_diario['sentiment_weighted'] > threshold_positive) |
    (sentimiento_diario['sentiment_weighted'] < threshold_negative)
)

dias_extremos = sentimiento_diario[sentimiento_diario['sentiment_extreme']]

print(f"📊 DÍAS CON SENTIMIENTO EXTREMO")
print("="*70)
print(f"\nUmbrales:")
print(f"   Media: {sentiment_mean:.4f}")
print(f"   Desviación estándar: {sentiment_std:.4f}")
print(f"   Umbral positivo (μ + 1.5σ): {threshold_positive:.4f}")
print(f"   Umbral negativo (μ - 1.5σ): {threshold_negative:.4f}")
print(f"\n🚨 Días extremos detectados: {len(dias_extremos)} ({len(dias_extremos)/len(sentimiento_diario)*100:.2f}%)")

📊 DÍAS CON SENTIMIENTO EXTREMO

Umbrales:
   Media: -0.1272
   Desviación estándar: 0.2840
   Umbral positivo (μ + 1.5σ): 0.2989
   Umbral negativo (μ - 1.5σ): -0.5532

🚨 Días extremos detectados: 269 (11.83%)


In [18]:
# Visualizar eventos extremos
fig = go.Figure()

# Sentimiento completo
fig.add_trace(go.Scatter(
    x=sentimiento_diario.index,
    y=sentimiento_diario['sentiment_weighted'],
    mode='lines',
    name='Sentimiento',
    line=dict(color='blue', width=1.5)
))

# Eventos extremos
fig.add_trace(go.Scatter(
    x=dias_extremos.index,
    y=dias_extremos['sentiment_weighted'],
    mode='markers',
    name='Eventos Extremos',
    marker=dict(color='red', size=10, symbol='star')
))

# Umbrales
fig.add_hline(y=threshold_positive, line_dash="dash", line_color="green", 
              annotation_text="Umbral +", annotation_position="right")
fig.add_hline(y=threshold_negative, line_dash="dash", line_color="red", 
              annotation_text="Umbral -", annotation_position="right")
fig.add_hline(y=0, line_dash="dot", line_color="gray")

fig.update_layout(
    title='🚨 Detección de Eventos de Sentimiento Extremo',
    xaxis_title='Fecha',
    yaxis_title='Sentimiento Ponderado',
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()

---

## 8️⃣ Ejemplos de Noticias por Sentimiento

In [19]:
# Mostrar ejemplos de cada categoría
print("📰 EJEMPLOS DE NOTICIAS POR SENTIMIENTO\n")

for sentiment_cat in ['Positivo', 'Neutral', 'Negativo']:
    print(f"\n{'='*70}")
    print(f"🔹 {sentiment_cat.upper()}")
    print(f"{'='*70}\n")
    
    ejemplos = df_noticias[
        df_noticias['sentiment'] == sentiment_cat
    ].nlargest(5, 'sentiment_score')
    
    for i, (idx, row) in enumerate(ejemplos.iterrows(), 1):
        print(f"{i}. {row['titulo']}")
        print(f"   Fecha: {row['fecha'].strftime('%Y-%m-%d')} | Confianza: {row['sentiment_score']:.3f}\n")

📰 EJEMPLOS DE NOTICIAS POR SENTIMIENTO


🔹 POSITIVO

1. FedEx Reports Better-Than-Expected Revenue as Residential Deliveries Surge
   Fecha: 2020-06-30 | Confianza: 0.959

2. Goldman Sachs Rides Deals to Improved Revenue, Earnings
   Fecha: 2018-10-16 | Confianza: 0.955

3. Gold Demand Rose 15% in the Second Quarter
   Fecha: 2016-08-11 | Confianza: 0.955

4. ECB Says Financing Conditions to Improve for Eurozone Firms
   Fecha: 2016-01-19 | Confianza: 0.954

5. Goldman CEO Lloyd Blankfein Sees Signs of Improving Economy
   Fecha: 2016-05-20 | Confianza: 0.954


🔹 NEUTRAL

1. Goldman and Global Atlantic to Issue, Sell Investment Products Based on Motif Indexes
   Fecha: 2016-05-20 | Confianza: 0.955

2. Warburg Sells Slickdeals to Goldman and Hearst
   Fecha: 2018-06-14 | Confianza: 0.951

3. What It Would Take for Amazon to Become UPS or FedEx
   Fecha: 2018-02-10 | Confianza: 0.951

4. Steel Heir Lists Long Island Gold Coast Estate for $29.9 Million
   Fecha: 2018-05-02 | Confianza: 0

---

## 9️⃣ Exportación de Datos con Sentimientos

In [20]:
# Preparar dataset de noticias con sentimientos
df_export_noticias = df_noticias[[
    'titulo', 'url', 'fecha',
    'sentiment', 'sentiment_label', 'sentiment_score',
    'sentiment_numeric', 'sentiment_weighted'
]].copy()

# Exportar
archivo_noticias_sentiment = DATA_PROCESADO_DIR / 'noticias_oro_con_sentimientos.csv'
df_export_noticias.to_csv(archivo_noticias_sentiment, index=False)

print(f"✅ Dataset de noticias con sentimientos exportado a:")
print(f"   {archivo_noticias_sentiment}")
print(f"\n📊 Variables incluidas: {list(df_export_noticias.columns)}")
print(f"\n📈 Total de noticias: {len(df_export_noticias):,}")

✅ Dataset de noticias con sentimientos exportado a:
   /home/els4nchez/Videos/TECH/unificacion/datos_procesados/noticias_oro_con_sentimientos.csv

📊 Variables incluidas: ['titulo', 'url', 'fecha', 'sentiment', 'sentiment_label', 'sentiment_score', 'sentiment_numeric', 'sentiment_weighted']

📈 Total de noticias: 18,776


In [21]:
# Exportar agregación diaria de sentimientos
archivo_sentiment_diario = DATA_PROCESADO_DIR / 'sentimientos_diarios.csv'
sentimiento_diario.to_csv(archivo_sentiment_diario)

print(f"✅ Sentimientos agregados diariamente exportados a:")
print(f"   {archivo_sentiment_diario}")
print(f"\n📊 Variables incluidas: {list(sentimiento_diario.columns)}")
print(f"\n📈 Total de días: {len(sentimiento_diario):,}")

✅ Sentimientos agregados diariamente exportados a:
   /home/els4nchez/Videos/TECH/unificacion/datos_procesados/sentimientos_diarios.csv

📊 Variables incluidas: ['sentiment_numeric', 'sentiment_weighted', 'sentiment_score', 'num_noticias', 'sentiment_ma_7', 'sentiment_ma_30', 'sentiment_extreme']

📈 Total de días: 2,273


In [22]:
# Guardar estadísticas del análisis de sentimientos
import json

sentiment_stats = {
    'modelo_usado': model_name,
    'total_noticias': len(df_noticias),
    'distribucion_sentimientos': {
        'Positivo': int(sentiment_counts.get('Positivo', 0)),
        'Neutral': int(sentiment_counts.get('Neutral', 0)),
        'Negativo': int(sentiment_counts.get('Negativo', 0))
    },
    'porcentajes': {
        'Positivo': float(sentiment_percentages.get('Positivo', 0)),
        'Neutral': float(sentiment_percentages.get('Neutral', 0)),
        'Negativo': float(sentiment_percentages.get('Negativo', 0))
    },
    'sentimiento_promedio': {
        'media': float(sentimiento_diario['sentiment_weighted'].mean()),
        'std': float(sentimiento_diario['sentiment_weighted'].std()),
        'min': float(sentimiento_diario['sentiment_weighted'].min()),
        'max': float(sentimiento_diario['sentiment_weighted'].max())
    },
    'eventos_extremos': {
        'total': int(len(dias_extremos)),
        'umbral_positivo': float(threshold_positive),
        'umbral_negativo': float(threshold_negative),
        'fechas_top_5_positivas': [str(d) for d in top_positive.index[:5]],
        'fechas_top_5_negativas': [str(d) for d in top_negative.index[:5]]
    }
}

archivo_stats = DATA_PROCESADO_DIR / 'estadisticas_sentimientos.json'

with open(archivo_stats, 'w', encoding='utf-8') as f:
    json.dump(sentiment_stats, f, indent=2, ensure_ascii=False)

print(f"✅ Estadísticas de sentimientos guardadas en: {archivo_stats}")

✅ Estadísticas de sentimientos guardadas en: /home/els4nchez/Videos/TECH/unificacion/datos_procesados/estadisticas_sentimientos.json


---

## 🔟 Resumen Ejecutivo

In [23]:
# Resumen ejecutivo
print("""
╔════════════════════════════════════════════════════════════════════════╗
║           💬 RESUMEN EJECUTIVO - ANÁLISIS DE SENTIMIENTOS              ║
╠════════════════════════════════════════════════════════════════════════╣
║                                                                        ║
║  ✅ ANÁLISIS COMPLETADO EXITOSAMENTE                                   ║
║                                                                        ║
║  🤖 MODELO UTILIZADO:                                                  ║
║                                                                        ║""")

print(f"║  • {model_name:66s} ║")
print(f"║  • Idioma: Inglés (CORRECCIÓN aplicada vs BETO español)           ║")
print(f"║  • Especialización: Textos financieros                            ║")

print("""
║                                                                        ║
║  📊 DISTRIBUCIÓN DE SENTIMIENTOS:                                      ║
║                                                                        ║""")

for sentiment in ['Positivo', 'Neutral', 'Negativo']:
    if sentiment in sentiment_counts.index:
        count = sentiment_counts[sentiment]
        pct = sentiment_percentages[sentiment]
        print(f"║  • {sentiment:10s}: {count:5d} noticias ({pct:5.2f}%)                          ║")

print(f"""
║                                                                        ║
║  📈 MÉTRICAS CLAVE:                                                    ║
║                                                                        ║
║  • Sentimiento promedio: {sentimiento_diario['sentiment_weighted'].mean():+.4f}                                  ║
║  • Desviación estándar: {sentimiento_diario['sentiment_weighted'].std():.4f}                                   ║
║  • Días con sentimiento extremo: {len(dias_extremos):3d} ({len(dias_extremos)/len(sentimiento_diario)*100:5.2f}%)                      ║
║  • Total de días analizados: {len(sentimiento_diario):,}                                ║
║                                                                        ║
║  🔧 PREPARACIÓN PARA CORRELACIÓN:                                      ║
║                                                                        ║
║  • Sentimientos agregados por día                                     ║
║  • Variables numéricas creadas (sentiment_numeric, weighted)          ║
║  • Medias móviles calculadas (7 y 30 días)                            ║
║  • Eventos extremos identificados                                     ║
║                                                                        ║
║  📂 ARCHIVOS GENERADOS:                                                ║
║                                                                        ║
║  • noticias_oro_con_sentimientos.csv (5,464 noticias)                 ║
║  • sentimientos_diarios.csv (agregación temporal)                     ║
║  • estadisticas_sentimientos.json (métricas y eventos)                ║
║                                                                        ║
║  ➡️  SIGUIENTE PASO:                                                   ║
║     Notebook 06 - Correlación y Causalidad (Outliers vs Sentimiento) ║
║                                                                        ║
╚════════════════════════════════════════════════════════════════════════╝
""")


╔════════════════════════════════════════════════════════════════════════╗
║           💬 RESUMEN EJECUTIVO - ANÁLISIS DE SENTIMIENTOS              ║
╠════════════════════════════════════════════════════════════════════════╣
║                                                                        ║
║  ✅ ANÁLISIS COMPLETADO EXITOSAMENTE                                   ║
║                                                                        ║
║  🤖 MODELO UTILIZADO:                                                  ║
║                                                                        ║
║  • ProsusAI/finbert                                                   ║
║  • Idioma: Inglés (CORRECCIÓN aplicada vs BETO español)           ║
║  • Especialización: Textos financieros                            ║

║                                                                        ║
║  📊 DISTRIBUCIÓN DE SENTIMIENTOS:                                      ║
║                                     